<a href="https://colab.research.google.com/github/phan-le-phu/Plate_detect_and_recognize/blob/master/Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/phan-le-phu/Plate_detect_and_recognize

Cloning into 'Plate_detect_and_recognize'...
remote: Enumerating objects: 210, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 210 (delta 11), reused 0 (delta 0), pack-reused 180
Receiving objects: 100% (210/210), 55.82 MiB | 24.13 MiB/s, done.
Resolving deltas: 100% (80/80), done.


In [ ]:
!unzip /content/car_long.zip

In [4]:
cd Plate_detect_and_recognize

/content/Plate_detect_and_recognize


In [5]:
# remove warning message
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# required library
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from local_utils import detect_lp
from os.path import splitext,basename
from keras.models import model_from_json
import glob
import math
from keras.models import model_from_json
from keras.preprocessing.image import load_img, img_to_array
from keras.applications.mobilenet_v2 import preprocess_input
from sklearn.preprocessing import LabelEncoder
from collections import Counter

In [6]:
def load_model(path):
    try:
        path = splitext(path)[0]
        with open('%s.json' % path, 'r') as json_file:
            model_json = json_file.read()
        model = model_from_json(model_json, custom_objects={})
        model.load_weights('%s.h5' % path)
        print("Loading model successfully...")
        return model
    except Exception as e:
        print(e)

In [7]:
wpod_net_path = "wpod-net.json"
wpod_net = load_model(wpod_net_path)

Loading model successfully...


In [8]:
# Load model architecture, weight and labels
json_file = open('MobileNets_character_recognition.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
model.load_weights("License_character_recognition_weight.h5")
print("[INFO] Model loaded successfully...")

labels = LabelEncoder()
labels.classes_ = np.load('license_character_classes.npy')
print("[INFO] Labels loaded successfully...")

[INFO] Model loaded successfully...
[INFO] Labels loaded successfully...


In [9]:
def preprocess_image(image_path,resize=False):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img / 255
    if resize:
        img = cv2.resize(img, (224,224))
    return img

def get_plate(image_path, Dmax=608, Dmin=256):
    vehicle = preprocess_image(image_path)
    ratio = float(max(vehicle.shape[:2])) / min(vehicle.shape[:2])
    side = int(ratio * Dmin)
    bound_dim = min(side, Dmax)
    _ , LpImg, _, cor = detect_lp(wpod_net, vehicle, bound_dim, lp_threshold=0.5)
    return vehicle, LpImg, cor

In [20]:
# Create sort_contours() function to grab the contour of each digit from left to right
def sort_contours(cnts, height, reverse = False):

    boundingBoxes = [cv2.boundingRect(c) for c in cnts]
    (cnts, boundingBoxes) = zip(*sorted(zip(cnts, boundingBoxes),
                                        key=lambda b: [1 if 1.2<=height / (b[1][1] + 1)<=3.0 else 0, b[1][0]], 
                                        reverse=reverse))
    return cnts

def predict_from_model(image,model,labels):
    image = cv2.resize(image,(80,80))
    image = np.stack((image,)*3, axis=-1)
    prediction = labels.inverse_transform([np.argmax(model.predict(image[np.newaxis,:]))])
    return prediction

In [18]:
def detect_and_recognize_plate(image_path):
  _, LpImg, _ = get_plate(image_path)

  for i in range(len(LpImg)):

      # Scales, calculates absolute values, and converts the result to 8-bit.
      plate_image = cv2.convertScaleAbs(LpImg[i], alpha=(255.0))
      
      # convert to grayscale and blur the image
      gray = cv2.cvtColor(plate_image, cv2.COLOR_BGR2GRAY)
      blur = cv2.GaussianBlur(gray,(7,7),0)
      
      # Applied inversed thresh_binary 
      binary = cv2.threshold(blur, 180, 255,
                          cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
      
      kernel3 = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
      thre_mor = cv2.morphologyEx(binary, cv2.MORPH_DILATE, kernel3)

      
      pad = cv2.copyMakeBorder(thre_mor, 1,1,1,1, cv2.BORDER_CONSTANT, value=255)
      h, w = pad.shape

      # create zeros mask 2 pixels larger in each dimension
      mask = np.zeros([h + 2, w + 2], np.uint8)

      # floodfill outer white border with black
      img_floodfill = cv2.floodFill(pad, mask, (0,0), 0, (5), (0), flags=8)[1]

      # remove border
      thre_mor = img_floodfill[1:h-1, 1:w-1]    

      cont, _  = cv2.findContours(thre_mor, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)


      # Initialize a list which will be used to append charater image
      crop_characters = []

      # define standard width and height of character
      digit_w, digit_h = 30, 60

      for c in sort_contours(cont, plate_image.shape[0]):
          (x, y, w, h) = cv2.boundingRect(c)
          ratio = h/w
          if 0.6<=ratio<=10: # Only select contour with defined ratio
              if 0.3<=h/plate_image.shape[0]<=0.9: # Select contour which has the height larger than 50% of the plate

                  # Sperate number and gibe prediction
                  curr_num = thre_mor[y:y+h,x:x+w]
                  curr_num = cv2.resize(curr_num, dsize=(digit_w, digit_h))
                  _, curr_num = cv2.threshold(curr_num, 220, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
                  crop_characters.append(curr_num)




      
      final_string = ''
      for i,character in enumerate(crop_characters):
          title = np.array2string(predict_from_model(character,model,labels))
          final_string+=title.strip("'[]")
      
      return final_string

In [12]:
import pandas as pd

In [27]:
df = pd.read_csv('/content/LABLE.csv')

In [28]:
df.head()

,filename,Platecharacters
0,0000_00532_b.jpg,59P164480
1,0000_02187_b.jpg,59E121500
2,0000_05696_b.jpg,59L206377
3,0000_06886_b.jpg,59C165331
4,0000_08244_b.jpg,59T108264


In [15]:
import time

In [30]:
start = time.time()
print(detect_and_recognize_plate('/content/Plate_detect_and_recognize/GreenParking/0000_00532_b.jpg'), time.time() - start)

59P166480 0.8915398120880127


In [32]:
name_files = df['filename']
characters = df['Platecharacters']

In [33]:
count = 0
for name, string in zip(name_files, characters):
  try:
    if string == detect_and_recognize_plate('/content/Plate_detect_and_recognize/GreenParking/' + name):
      count += 1
    else:
      print(name)
  except:
    print(f"Runtime error {name}")

0000_00532_b.jpg
0002_02183_b.jpg
0003_02063_b.jpg
0003_07398_b.jpg
0004_06801_b.jpg
0005_00490_b.jpg
0005_00512_b.jpg
0005_02171_b.jpg
0006_06035_b.jpg
0006_06797_b.jpg
0007_05159_b.jpg
0009_00490_b.jpg
0009_02194_b.jpg
0009_05325_b.jpg
0010_02063_b.jpg
0011_00515_b.jpg
0012_04539_b.jpg
0013_05647_b.jpg
0014_02176_b.jpg
0019_06895_b.jpg
0020_02063_b.jpg
0020_07156_b.jpg
0021_06361_b.jpg
0022_01753_b.jpg
0028_00196_b.jpg
0028_08102_b.jpg
0031_07285_b.jpg
0038_01762_b.jpg
0039_01689_b.jpg
0041_00507_b.jpg
0041_00697_b.jpg
0041_04542_b.jpg
0042_01746_b.jpg
0042_05741_b.jpg
0043_01133_b.jpg
0045_01674_b.jpg
0046_00961_b.jpg
0046_06241_b.jpg
0047_00717_b.jpg
0047_00970_b.jpg
0047_01152_b.jpg
0047_03046_b.jpg
0048_01689_b.jpg
0048_01752_b.jpg
0049_03047_b.jpg
0049_04676_b.jpg
0050_01152_b.jpg
0051_00959_b.jpg
0052_05252_b.jpg
0054_03047_b.jpg
0054_04076_b.jpg
0054_05705_b.jpg
0054_06669_b.jpg
0055_01981_b.jpg
0056_01595_b.jpg
0056_05671_b.jpg
0057_04356_b.jpg
0057_06324_b.jpg
0058_00594_b.j

In [34]:
print(count)

93


In [ ]:
!unzip /content/GreenParking.zip